# Notebook for step 4 of AML Project
This notebook is created for fine tuning the OpenShape model. We have used code from the following repositories:

Open shape:
@misc{liu2023openshape,
      title={OpenShape: Scaling Up 3D Shape Representation Towards Open-World Understanding},
      author={Minghua Liu and Ruoxi Shi and Kaiming Kuang and Yinhao Zhu and Xuanlin Li and Shizhong Han and Hong Cai and Fatih Porikli and Hao Su},
      year={2023},
      eprint={2305.10764},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}

3DOS:
@inproceedings{
alliegro2022towards,
title={Towards Open Set 3D Learning: Benchmarking and Understanding Semantic Novelty Detection on Pointclouds},
author={Antonio Alliegro and Francesco Cappio Borlino and Tatiana Tommasi},
booktitle={Thirty-sixth Conference on Neural Information Processing Systems Datasets and Benchmarks Track},
year={2022},
url={https://openreview.net/forum?id=X2dHozbd1at}
}

# Clone git repos to environment

In [3]:

!git clone https://github.com/Colin97/OpenShape_code.git
# Make sure you have git-lfs installed (https://git-lfs.com)
!git lfs install
!git clone https://huggingface.co/spaces/OpenShape/openshape-demo

# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
!GIT_LFS_SKIP_SMUDGE=1
#!pip install -e .
!git clone https://huggingface.co/OpenShape/openshape-demo-support


fatal: destination path 'OpenShape_code' already exists and is not an empty directory.
Git LFS initialized.
fatal: destination path 'openshape-demo' already exists and is not an empty directory.
fatal: destination path 'openshape-demo-support' already exists and is not an empty directory.


# Install dependencies
Most dependencies are already satisfied by default in google colab

In [4]:
!pip install huggingface_hub wandb omegaconf torch_redstone einops tqdm open3d dgl timm
!pip install --upgrade https://github.com/unlimblue/KNN_CUDA/releases/download/0.2/KNN_CUDA-0.2-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/

# Log in to Huggingface

In [5]:
from google.colab import output
output.enable_custom_widget_manager()

In [6]:
from huggingface_hub import login
login()

# Get the OpenShape model from Huggingface

In [1]:
%cd /content/openshape-demo-support
!pip install -e .
%cd /content

/content/openshape-demo-support
Obtaining file:///content/openshape-demo-support
  Preparing metadata (setup.py) ... done
  Attempting uninstall: openshape
    Found existing installation: openshape 0.1
    Uninstalling openshape-0.1:
      Successfully uninstalled openshape-0.1
  Running setup.py develop for openshape
/content


In [2]:
# Restart run time if this cell fails and try again
import openshape
model = openshape.load_pc_encoder('openshape-pointbert-vitg14-rgb')


model.eval()

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.pt:   0%|          | 0.00/388M [00:00<?, ?B/s]

Projected(
  (ppat): PointPatchTransformer(
    (sa): PointNetSetAbstraction(
      (mlp_convs): ModuleList(
        (0): Conv2d(9, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      )
      (mlp_bns): ModuleList(
        (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lift): Sequential(
      (0): Conv1d(259, 512, kernel_size=(1,), stride=(1,))
      (1): Lambda()
      (2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (transformer): Transformer(
      (layers): ModuleList(
        (0-11): 12 x ModuleList(
          (0): PreNorm(
            (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (fn): Attention(
              (attend): Softmax(dim=-1)
              (dropout):

# Function for fitting pointclouds to input dimentions for OpenShape

In [3]:
import numpy as np
import open3d as o3d
import random
import torch
from OpenShape_code.src.utils.data import normalize_pc

def load_ply(file_name, num_points=10000, y_up=True):
    pcd = o3d.io.read_point_cloud(file_name)  # Read the point cloud
    xyz = np.asarray(pcd.points)  # Get xyz coordinates
    rgb = np.asarray(pcd.colors)  # Get rgb colors
    n = xyz.shape[0]

    # Sample num_points points if necessary
    if n > num_points:
        idx = random.sample(range(n), num_points)
        xyz = xyz[idx]
        rgb = rgb[idx]
    elif n < num_points:
        print(f"Warning: requested {num_points} points, but file has only {n} points.", file=sys.stderr)

    # Adjust orientation by swapping y and z if requested
    if y_up:
        xyz[:, [1, 2]] = xyz[:, [2, 1]]

    # Normalize the point cloud coordinates
    xyz_normalized = normalize_pc(xyz)

    # Handle cases where rgb might be missing
    if rgb.size == 0:
        rgb = np.ones_like(xyz_normalized) * 0.4  # Default to a constant color if missing

    # Concatenate xyz with rgb
    features = np.concatenate([xyz_normalized, rgb], axis=1)

    # Convert to PyTorch tensors
    xyz_tensor = torch.from_numpy(xyz_normalized).float()
    features_tensor = torch.from_numpy(features).float()

    # Add batch dimention to fit as single input to OpenShape model
    features_tensor = features_tensor.unsqueeze(0)  # Adds a batch dimension, making it [1, N, 6]
    features_tensor = features_tensor.transpose(1, 2)  # Transposes to get [1, 6, N], matching the expected [B, C, N] format

    # Returning tensors instead of ME-specific batched coordinates
    return xyz_tensor, features_tensor

def filter_for_OpenShape(pointcloud, num_points=10000, y_up=True):
    xyz = np.asarray(pointcloud.points)  # Get xyz coordinates
    rgb = np.asarray(pointcloud.colors)  # Get rgb colors
    n = xyz.shape[0]

    # Sample num_points points if necessary
    if n > num_points:
        idx = random.sample(range(n), num_points)
        xyz = xyz[idx]
        rgb = rgb[idx]
    elif n < num_points:
        print(f"Warning: requested {num_points} points, but file has only {n} points.", file=sys.stderr)

    # Adjust orientation by swapping y and z if requested
    if y_up:
        xyz[:, [1, 2]] = xyz[:, [2, 1]]

    # Normalize the point cloud coordinates
    xyz_normalized = normalize_pc(xyz)

    # Handle cases where rgb might be missing
    if rgb.size == 0:
        rgb = np.ones_like(xyz_normalized) * 0.4  # Default to a constant color if missing

    # Concatenate xyz with rgb
    features = np.concatenate([xyz_normalized, rgb], axis=1)

    # Convert to PyTorch tensors
    xyz_tensor = torch.from_numpy(xyz_normalized).float()
    features_tensor = torch.from_numpy(features).float()

    # Add batch dimention to fit as single input to OpenShape model
    features_tensor = features_tensor.unsqueeze(0)  # Adds a batch dimension, making it [1, N, 6]
    features_tensor = features_tensor.transpose(1, 2)  # Transposes to get [1, 6, N], matching the expected [B, C, N] format

    # Returning tensors instead of ME-specific batched coordinates
    return xyz_tensor, features_tensor


# Test that installation and environment works

If a feature vector is printed it should all be A OK!

In [ ]:
xyz, feat = load_ply("/content/OpenShape_code/demo/owl.ply")

device = torch.device("cpu")
model = model.to(device)
feat = feat.to(device)

output = model.forward(feat)

print(output)


# Clone 3DOS repo

In [5]:
# Clone 3D_OS repo from github
!git clone https://github.com/antoalli/3D_OS.git
!cd 3D_OS && chmod +x download_data.sh && ./download_data.sh
!pip install h5py protobuf lmdb msgpack-numpy ninja scikit-learn


Cloning into '3D_OS'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 152 (delta 51), reused 142 (delta 44), pack-reused 0
Receiving objects: 100% (152/152), 129.35 KiB | 7.19 MiB/s, done.
Resolving deltas: 100% (51/51), done.
============Downloading ShapeNetCore resampled in 
--2024-02-19 13:08:00--  https://www.dropbox.com/s/oa3qbujpugw4d43/sncore_fps_4096.tar?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/oa3qbujpugw4d43/sncore_fps_4096.tar [following]
--2024-02-19 13:08:00--  https://www.dropbox.com/s/dl/oa3qbujpugw4d43/sncore_fps_4096.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaed4be7fbd1c235be008247765.dl

# Using SR1 as ID create dataloaders for train, ID, OOD1 and OOD2.

The datasets we use are the same as the datasets used in 3DOS. We also use the code given in the 3DOS paper to create the dataloaders.


Citation:
\@inproceedings{
alliegro2022towards,
title={Towards Open Set 3D Learning: Benchmarking and Understanding Semantic Novelty Detection on Pointclouds},
author={Antonio Alliegro and Francesco Cappio Borlino and Tatiana Tommasi},
booktitle={Thirty-sixth Conference on Neural Information Processing Systems Datasets and Benchmarks Track},
year={2022},
url={https://openreview.net/forum?id=X2dHozbd1at}
}

In [ ]:
# Change folder to 3D_OS to get the same dataloaders
%cd /content/3D_OS

import sys
import os
import warnings
import numpy as np

sys.path.append(os.getcwd())
import os.path as osp
import time
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.utils import *
from utils.dist import *
# noinspection PyUnresolvedReferences
from utils.data_utils import H5_Dataset
#from datasets.modelnet import *
from datasets.scanobject import *
from models.classifiers import Classifier
from utils.ood_utils import get_confidence, eval_ood_sncore, iterate_data_odin, \
    iterate_data_energy, iterate_data_gradnorm, iterate_data_react, estimate_react_thres, print_ood_output, \
    get_penultimate_feats, get_network_output
import wandb
from base_args import add_base_args
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from models.common import convert_model_state, logits_entropy_loss
from models.ARPL_utils import Generator, Discriminator
from classifiers.common import train_epoch_cla, train_epoch_rsmix_exposure, train_epoch_cs
from classifiers.trainer_ddp_cla_md import get_args, load_yaml, get_md_eval_loaders

# Code from 3DOS repo classifiers/trainer_ddp_cla_md.py line 192-478

#  python -m torch.distributed.launch --nproc_per_node=1 Failure_Analysis/custom\
# tests/evaluating_PointNet_cosine_MLS.py --config cfgs/pn2-msg.yaml
# --exp_name PN2_cosine_SR1 --src SR1 --loss cosine -mode eval
# --ckpt_path outputs/PN2_cosine_SR1/models/model_best.pth


# Code from 3DOS repo classifiers/trainer_ddp_cla_md.py line 468-478
def eval_ood_md2sonn(opt, config):
    print(f"Arguments: {opt}")
    set_random_seed(opt.seed)

    dataloader_config = {
        'batch_size': opt.batch_size, 'drop_last': False, 'shuffle': False,
        'num_workers': opt.num_workers, 'sampler': None, 'worker_init_fn': init_np_seed}

    # whole evaluation is done on ScanObject RW data
    sonn_args = {
        'data_root': opt.data_root,
        'sonn_split': opt.sonn_split,
        'h5_file': opt.sonn_h5_name,
        'split': 'all',  # we use both training (unused) and test samples during evaluation
        'num_points': opt.num_points_test,  # default: use all 2048 sonn points to avoid sampling randomicity
        'transforms': None  # no augmentation applied at inference time
    }

    train_loader, _ = get_md_eval_loaders(opt)
    if opt.src == 'SR1':
        print("Src is SR1\n")
        id_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet1", **sonn_args), **dataloader_config)
        ood1_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet2", **sonn_args), **dataloader_config)
    elif opt.src == 'SR2':
        print("Src is SR2\n")
        id_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet2", **sonn_args), **dataloader_config)
        ood1_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet1", **sonn_args), **dataloader_config)
    else:
        raise ValueError(f"OOD evaluation - wrong src: {opt.src}")

    ood2_loader = DataLoader(ScanObject(class_choice="sonn_ood_common", **sonn_args), **dataloader_config)

    return train_loader, id_loader, ood1_loader, ood2_loader

# Ugly hacks to change the minimum amount of code hehe
class Options:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

opt_dict = {
    "apply_fix_cellphone": True,
    "augm_set": 'rw',
    "batch_size": 1,
    "checkpoints_dir": 'outputs',
    "ckpt_path": 'outputs/PN2_cosine_SR1/models/model_best.pth',
    "config": 'cfgs/pn2-msg.yaml',
    "corruption": None,
    "cs": False,
    "cs_beta": 0.1,
    "cs_gan_lr": 0.0002,
    "data_root": "/content/3D_OS/3D_OS_release_data",
    "epochs": 250,
    "eval_step": 1,
    "exp_name": 'PN2_cosine_SR1',
    "grad_norm_clip": -1,
    "local_rank": 0,
    "loss": 'cosine',
    "num_points": 10000,
    "num_points_test": 10000,
    "num_workers": 6,
    "resume": None,
    "save_feats": None,
    "save_step": 10,
    "script_mode": 'eval',
    "seed": 1,
    "sonn_h5_name": 'objectdataset.h5',
    "sonn_split": 'main_split',
    "src": 'SR1',
    "tar1": 'none',
    "tar2": 'none',
    "use_amp": False,
    "use_sync_bn": False,
    "wandb_group": 'md-2-sonn-augmCorr',
    "wandb_name": None,
    "wandb_proj": 'benchmark-3d-ood-cla'
}

opt = Options(opt_dict)

config = {'optimizer': {'type': 'adam',
                        'skip_wd': [],
                        'weight_decay': 0.0001,
                        'kwargs': {'lr': 0.001}
                        },
          'scheduler': {'type': 'CosLR',
                        'kwargs': {'t_initial': 250,
                                   'cycle_limit': 1,
                                   'lr_min': 1e-05
                                   }
                        },
          'model': {'ENCO_NAME': 'pn2-msg',
                    'dropout': 0.5,
                    'cla_input_dim': 1024,
                    'act': 'relu'
                    }
          }

print(opt)
print(config)

train_loader, src_loader, tar1_loader, tar2_loader = eval_ood_md2sonn(opt, config)

%cd /content


# Test if dataloaders work

In [7]:
# Test if the data loaders work
import plotly.graph_objects as go

for i, (data, label) in enumerate(src_loader):
    points = data
    points = points.reshape(-1, 3)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    fig = go.Figure(
      data=[
          go.Scatter3d(
            x=points[:,0],
            y=points[:,1],
            z=points[:,2],
            mode='markers',
          )
      ],
      layout=dict(
            scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
      )
    )
    fig.show()
    break


# Run data through the OpenShape model

In [ ]:
# Run data through OpenShape
from tqdm import tqdm
device = torch.device("cpu")
model = model.to(device)

%cd /content
save_dir = "OpenShape_outputs/SR1"
if not os.path.exists(save_dir):
    print(f"Creating directory: {save_dir}")
    os.makedirs(save_dir)

print("Running training data through the OpenShape model")
train_feats = []
for i, (data, label) in tqdm(enumerate(train_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    train_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/train_feats.npy"
np.save(outfile, np.array(train_feats))
print("Saved data to " + outfile)

print("Running ID data through the OpenShape model")
src_feats = []
for i, (data, label) in tqdm(enumerate(src_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    src_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/src_feats"
np.save(outfile, np.array(src_feats))
print("Saved data to " + outfile)

print("Rinning OOD1 data throug the OpenShape model")
tar1_feats = []
for i, (data, label) in tqdm(enumerate(tar1_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    tar1_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/tar1_feats"
np.save(outfile, np.array(tar1_feats))
print("Saved data to " + outfile)

print("Rinning OOD2 data throug the OpenShape model")
tar2_feats = []
for i, (data, label) in tqdm(enumerate(tar2_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    tar2_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/tar2_feats"
np.save(outfile, np.array(tar2_feats))
print("Saved data to " + outfile)

In [12]:
old_train_feats = train_feats
old_src_feats = src_feats
old_tar1_feats = tar1_feats
old_tar2_feats = tar2_feats

In [14]:
# Load np arrays and convert to torch tensors

train_feats = np.load("OpenShape_outputs/SR1/train_feats.npy")
src_feats = np.load("OpenShape_outputs/SR1/src_feats.npy")
tar1_feats = np.load("OpenShape_outputs/SR1/tar1_feats.npy")
tar2_feats = np.load("OpenShape_outputs/SR1/tar2_feats.npy")

train_feats = torch.from_numpy(train_feats)
src_feats = torch.from_numpy(src_feats)
tar1_feats = torch.from_numpy(tar1_feats)
tar2_feats = torch.from_numpy(tar2_feats)

In [15]:
# Example NumPy array and target number of tensors
#x_elements = old_tar2_feats  # Create an array of 24 elements
#y_tensors = 847  # Want to divide into 4 tensors

# Ensure x_elements can be evenly divided into y_tensors
#if len(x_elements) % y_tensors != 0:
#    raise ValueError("The number of elements in the array must be divisible by y_tensors")

# Split the array into y parts
#split_arrays = np.array_split(x_elements, y_tensors)

# Convert each part into a tensor and stack them into a single tensor
#tensors_list = [torch.tensor(part) for part in split_arrays]
#final_tar2_feats = torch.stack(tensors_list)

#print(final_tar2_feats)

In [16]:
#train_feats = final_train_feats
#src_feats = final_src_feats
#tar1_feats = final_tar1_feats
#tar2_feats = final_tar2_feats

In [17]:

from knn_cuda import KNN
knn = KNN(k=1, transpose_mode=True)

device = torch.device("cuda")
train_feats = train_feats.to(device)
src_feats = src_feats.to(device)
tar1_feats = tar1_feats.to(device)
tar2_feats = tar2_feats.to(device)

################################################
print("Euclidean distances in a non-normalized space:")
# eucl distance in a non-normalized space
src_dist, src_ids = knn(train_feats.unsqueeze(0), src_feats.unsqueeze(0))
src_dist = src_dist.squeeze().cpu()
src_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_scores = 1 / src_dist
#src_pred = np.asarray([train_labels[i] for i in src_ids])  # pred is label of nearest training sample

# OOD tar1
tar1_dist, _ = knn(train_feats.unsqueeze(0), tar1_feats.unsqueeze(0))
tar1_dist = tar1_dist.squeeze().cpu()
tar1_scores = 1 / tar1_dist

# OOD tar2
tar2_dist, _ = knn(train_feats.unsqueeze(0), tar2_feats.unsqueeze(0))
tar2_dist = tar2_dist.squeeze().cpu()
tar2_scores = 1 / tar2_dist

eval_ood_sncore(
    scores_list=[src_scores, tar1_scores, tar2_scores],
    preds_list=[None, None, None],  # [src_pred, None, None],
    labels_list=[None, None, None],  # [src_labels, None, None],
    src_label=1  # confidence should be higher for ID samples
)


Euclidean distances in a non-normalized space:
AUROC - Src label: 1, Tar label: 0
SRC->TAR1:      AUROC: 0.4911, FPR95: 0.9619, AUPR_IN: 0.6130, AUPR_OUT: 0.3723
SRC->TAR2:      AUROC: 0.4656, FPR95: 0.9740, AUPR_IN: 0.5793, AUPR_OUT: 0.3707
SRC->TAR1+TAR2: AUROC: 0.4779, FPR95: 0.9682, AUPR_IN: 0.4240, AUPR_OUT: 0.5407
to spreadsheet: 0.49109652759520295,0.9619289340101523,0.6130192027508414,0.37232793731694475,0.46556066172147303,0.974025974025974,0.5792628549927861,0.3706976149858082,0.47786785579394964,0.9681957186544342,0.42404305308054707,0.5407019558215677


(-1,
 -1,
 {'fpr_at_95_tpr': 0.9619289340101523,
  'detection_error': 0.6078318716012313,
  'auroc': 0.49109652759520295,
  'aupr_in': 0.6130192027508414,
  'aupr_out': 0.37232793731694475},
 {'fpr_at_95_tpr': 0.974025974025974,
  'detection_error': 0.5968621798459786,
  'auroc': 0.46556066172147303,
  'aupr_in': 0.5792628549927861,
  'aupr_out': 0.3706976149858082},
 {'fpr_at_95_tpr': 0.9681957186544342,
  'detection_error': 0.4341756472833574,
  'auroc': 0.47786785579394964,
  'aupr_in': 0.42404305308054707,
  'aupr_out': 0.5407019558215677})

# Using SR2 as ID create dataloaders for train, ID, OOD1 and OOD2.

The datasets we use are the same as the datasets used in 3DOS. We also use the code given in the 3DOS paper to create the dataloaders.


Citation:
\@inproceedings{
alliegro2022towards,
title={Towards Open Set 3D Learning: Benchmarking and Understanding Semantic Novelty Detection on Pointclouds},
author={Antonio Alliegro and Francesco Cappio Borlino and Tatiana Tommasi},
booktitle={Thirty-sixth Conference on Neural Information Processing Systems Datasets and Benchmarks Track},
year={2022},
url={https://openreview.net/forum?id=X2dHozbd1at}
}

In [ ]:
# Clone 3D_OS repo from github
!git clone https://github.com/antoalli/3D_OS.git
!cd 3D_OS && chmod +x download_data.sh && ./download_data.sh
!pip install h5py protobuf lmdb msgpack-numpy ninja scikit-learn

In [ ]:
# Change folder to 3D_OS to get the same dataloaders
%cd /content/3D_OS

import sys
import os
import warnings
import numpy as np

sys.path.append(os.getcwd())
import os.path as osp
import time
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.utils import *
from utils.dist import *
# noinspection PyUnresolvedReferences
from utils.data_utils import H5_Dataset
#from datasets.modelnet import *
from datasets.scanobject import *
from models.classifiers import Classifier
from utils.ood_utils import get_confidence, eval_ood_sncore, iterate_data_odin, \
    iterate_data_energy, iterate_data_gradnorm, iterate_data_react, estimate_react_thres, print_ood_output, \
    get_penultimate_feats, get_network_output
import wandb
from base_args import add_base_args
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from models.common import convert_model_state, logits_entropy_loss
from models.ARPL_utils import Generator, Discriminator
from classifiers.common import train_epoch_cla, train_epoch_rsmix_exposure, train_epoch_cs
from classifiers.trainer_ddp_cla_md import get_args, load_yaml, get_md_eval_loaders

# Code from 3DOS repo classifiers/trainer_ddp_cla_md.py line 192-478

#  python -m torch.distributed.launch --nproc_per_node=1 Failure_Analysis/custom\
# tests/evaluating_PointNet_cosine_MLS.py --config cfgs/pn2-msg.yaml
# --exp_name PN2_cosine_SR1 --src SR1 --loss cosine -mode eval
# --ckpt_path outputs/PN2_cosine_SR1/models/model_best.pth


# Code from 3DOS repo classifiers/trainer_ddp_cla_md.py line 468-478
def eval_ood_md2sonn(opt, config):
    print(f"Arguments: {opt}")
    set_random_seed(opt.seed)

    dataloader_config = {
        'batch_size': opt.batch_size, 'drop_last': False, 'shuffle': False,
        'num_workers': opt.num_workers, 'sampler': None, 'worker_init_fn': init_np_seed}

    # whole evaluation is done on ScanObject RW data
    sonn_args = {
        'data_root': opt.data_root,
        'sonn_split': opt.sonn_split,
        'h5_file': opt.sonn_h5_name,
        'split': 'all',  # we use both training (unused) and test samples during evaluation
        'num_points': opt.num_points_test,  # default: use all 2048 sonn points to avoid sampling randomicity
        'transforms': None  # no augmentation applied at inference time
    }

    train_loader, _ = get_md_eval_loaders(opt)
    if opt.src == 'SR1':
        print("Src is SR1\n")
        id_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet1", **sonn_args), **dataloader_config)
        ood1_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet2", **sonn_args), **dataloader_config)
    elif opt.src == 'SR2':
        print("Src is SR2\n")
        id_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet2", **sonn_args), **dataloader_config)
        ood1_loader = DataLoader(ScanObject(class_choice="sonn_2_mdSet1", **sonn_args), **dataloader_config)
    else:
        raise ValueError(f"OOD evaluation - wrong src: {opt.src}")

    ood2_loader = DataLoader(ScanObject(class_choice="sonn_ood_common", **sonn_args), **dataloader_config)

    return train_loader, id_loader, ood1_loader, ood2_loader

class Options:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

opt_dict = {
    "apply_fix_cellphone": True,
    "augm_set": 'rw',
    "batch_size": 1,
    "checkpoints_dir": 'outputs',
    "ckpt_path": 'outputs/PN2_cosine_SR1/models/model_best.pth',
    "config": 'cfgs/pn2-msg.yaml',
    "corruption": None,
    "cs": False,
    "cs_beta": 0.1,
    "cs_gan_lr": 0.0002,
    "data_root": "/content/3D_OS/3D_OS_release_data",
    "epochs": 250,
    "eval_step": 1,
    "exp_name": 'PN2_cosine_SR1',
    "grad_norm_clip": -1,
    "local_rank": 0,
    "loss": 'cosine',
    "num_points": 10000,
    "num_points_test": 10000,
    "num_workers": 6,
    "resume": None,
    "save_feats": None,
    "save_step": 10,
    "script_mode": 'eval',
    "seed": 1,
    "sonn_h5_name": 'objectdataset.h5',
    "sonn_split": 'main_split',
    "src": 'SR2',
    "tar1": 'none',
    "tar2": 'none',
    "use_amp": False,
    "use_sync_bn": False,
    "wandb_group": 'md-2-sonn-augmCorr',
    "wandb_name": None,
    "wandb_proj": 'benchmark-3d-ood-cla'
}

opt = Options(opt_dict)

config = {'optimizer': {'type': 'adam',
                        'skip_wd': [],
                        'weight_decay': 0.0001,
                        'kwargs': {'lr': 0.001}
                        },
          'scheduler': {'type': 'CosLR',
                        'kwargs': {'t_initial': 250,
                                   'cycle_limit': 1,
                                   'lr_min': 1e-05
                                   }
                        },
          'model': {'ENCO_NAME': 'pn2-msg',
                    'dropout': 0.5,
                    'cla_input_dim': 1024,
                    'act': 'relu'
                    }
          }

print(opt)
print(config)

train_loader, src_loader, tar1_loader, tar2_loader = eval_ood_md2sonn(opt, config)

%cd /content


# Run data through the OpenShape model

In [ ]:
# Run data through OpenShape
from tqdm import tqdm
device = torch.device("cpu")
model = model.to(device)

%cd /content
save_dir = "OpenShape_outputs/SR2"
if not os.path.exists(save_dir):
    print(f"Creating directory: {save_dir}")
    os.makedirs(save_dir)

print("Running training data through the OpenShape model")
train_feats = []
for i, (data, label) in tqdm(enumerate(train_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    train_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/train_feats.npy"
np.save(outfile, np.array(train_feats))
print("Saved data to " + outfile)

In [ ]:


print("Running ID data through the OpenShape model")
src_feats = []
for i, (data, label) in tqdm(enumerate(src_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    src_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/src_feats"
np.save(outfile, np.array(src_feats))
print("Saved data to " + outfile)

print("Rinning OOD1 data throug the OpenShape model")
tar1_feats = []
for i, (data, label) in tqdm(enumerate(tar1_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    tar1_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/tar1_feats"
np.save(outfile, np.array(tar1_feats))
print("Saved data to " + outfile)

print("Rinning OOD2 data throug the OpenShape model")
tar2_feats = []
for i, (data, label) in tqdm(enumerate(tar2_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    tar2_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/tar2_feats"
np.save(outfile, np.array(tar2_feats))
print("Saved data to " + outfile)

In [ ]:
print("Rinning OOD2 data throug the OpenShape model")
tar2_feats = []
for i, (data, label) in tqdm(enumerate(tar2_loader)):
    points = data
    points = points.reshape(-1, 3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    xyz, feat = filter_for_OpenShape(pcd)
    feat = feat.to(device)

    output_feat = model.forward(feat)

    tar2_feats.append(output_feat.detach().numpy())

outfile = save_dir + "/tar2_feats"
np.save(outfile, np.array(tar2_feats))
print("Saved data to " + outfile)

In [26]:
# Save inferences to drive
!cp -r "/content/OpenShape_outputs" "/content/drive/MyDrive/Skole/Advanced machine learning project"

# Load inferences
!cp -r "/content/drive/MyDrive/Skole/Advanced machine learning project/OpenShape_outputs" "/content"

In [27]:
# Load np arrays and convert to torch tensors

train_feats = np.load("OpenShape_outputs/SR2/train_feats.npy")
src_feats = np.load("OpenShape_outputs/SR2/src_feats.npy")
tar1_feats = np.load("OpenShape_outputs/SR2/tar1_feats.npy")
tar2_feats = np.load("OpenShape_outputs/SR2/tar2_feats.npy")

train_feats = np.squeeze(train_feats, axis=1)
src_feats = np.squeeze(src_feats, axis=1)
tar1_feats = np.squeeze(tar1_feats, axis=1)
tar2_feats = np.squeeze(tar2_feats, axis=1)

train_feats = torch.from_numpy(train_feats)
src_feats = torch.from_numpy(src_feats)
tar1_feats = torch.from_numpy(tar1_feats)
tar2_feats = torch.from_numpy(tar2_feats)

In [28]:

from knn_cuda import KNN
knn = KNN(k=1, transpose_mode=True)

device = torch.device("cuda")
train_feats = train_feats.to(device)
src_feats = src_feats.to(device)
tar1_feats = tar1_feats.to(device)
tar2_feats = tar2_feats.to(device)

################################################
print("Euclidean distances in a non-normalized space:")
# eucl distance in a non-normalized space
src_dist, src_ids = knn(train_feats.unsqueeze(0), src_feats.unsqueeze(0))
src_dist = src_dist.squeeze().cpu()
src_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_scores = 1 / src_dist
#src_pred = np.asarray([train_labels[i] for i in src_ids])  # pred is label of nearest training sample

# OOD tar1
tar1_dist, _ = knn(train_feats.unsqueeze(0), tar1_feats.unsqueeze(0))
tar1_dist = tar1_dist.squeeze().cpu()
tar1_scores = 1 / tar1_dist

# OOD tar2
tar2_dist, _ = knn(train_feats.unsqueeze(0), tar2_feats.unsqueeze(0))
tar2_dist = tar2_dist.squeeze().cpu()
tar2_scores = 1 / tar2_dist

eval_ood_sncore(
    scores_list=[src_scores, tar1_scores, tar2_scores],
    preds_list=[None, None, None],  # [src_pred, None, None],
    labels_list=[None, None, None],  # [src_labels, None, None],
    src_label=1  # confidence should be higher for ID samples
)


Euclidean distances in a non-normalized space:
AUROC - Src label: 1, Tar label: 0
SRC->TAR1:      AUROC: 0.6378, FPR95: 0.7976, AUPR_IN: 0.5013, AUPR_OUT: 0.7569
SRC->TAR2:      AUROC: 0.5238, FPR95: 0.9410, AUPR_IN: 0.5279, AUPR_OUT: 0.5344
SRC->TAR1+TAR2: AUROC: 0.5919, FPR95: 0.8554, AUPR_IN: 0.3504, AUPR_OUT: 0.8035
to spreadsheet: 0.6378405161081562,0.797609561752988,0.5012826000491963,0.7568587052361571,0.5237670727979911,0.9409681227863046,0.5279115998487289,0.5343680824053896,0.5918746709684275,0.8553758325404377,0.35043461746757487,0.8034728931350148


(-1,
 -1,
 {'fpr_at_95_tpr': 0.797609561752988,
  'detection_error': 0.33804663672236474,
  'auroc': 0.6378405161081562,
  'aupr_in': 0.5012826000491963,
  'aupr_out': 0.7568587052361571},
 {'fpr_at_95_tpr': 0.9409681227863046,
  'detection_error': 0.4786812532099425,
  'auroc': 0.5237670727979911,
  'aupr_in': 0.5279115998487289,
  'aupr_out': 0.5343680824053896},
 {'fpr_at_95_tpr': 0.8553758325404377,
  'detection_error': 0.25801164899406104,
  'auroc': 0.5918746709684275,
  'aupr_in': 0.35043461746757487,
  'aupr_out': 0.8034728931350148})

In [29]:
# Run this command to save data to google drive
#!cp -r /content/OpenShape_outputs "/content/drive/MyDrive/Skole/Advanced machine learning project"